# ETL — Accidents de la route

Ce notebook montre, pas à pas, comment :
1) se connecter à l’API publique Opendatasoft,
2) récupérer un petit échantillon,
3) paginer pour extraire un volume plus grand,
4) sauvegarder les données brutes en CSV,
5) poser les bases du nettoyage (à faire en équipe).

> **Pourquoi ce format ?**  
> Un notebook est idéal pour apprendre : on alterne **explications** (Markdown) et **code** (Python), et on voit les résultats immédiatement.

In [ ]:
# Imports
import pandas as pd
import requests
import time
from pathlib import Path

# Dossiers de sortie
RAW_DIR = Path("../data/raw")
CLEAN_DIR = Path("../data/cleaned")
RAW_DIR.mkdir(parents=True, exist_ok=True)
CLEAN_DIR.mkdir(parents=True, exist_ok=True)

# Endpoint API (dataset Opendatasoft)
BASE_URL = "https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/accidents-corporels-de-la-circulation-millesime/records/?limit=10&offset=0"
# Colonnes utiles pour débuter
FIELDS = [
    "numac",
    "departement",
    "codeinsee",
    "lumiere",
    "condition_atmospherique",
    "categorie_de_route",
    "type_de_collision",
    "lat",
    # --- Temps : l'un des deux blocs sera présent ---
    "date",                 # si le dataset fournit directement la date
    "an", "mois", "jour", "hrmn"  # sinon, on reconstruit
]




In [25]:
def fetch_page(offset: int = 0, limit: int = 1000) -> list[dict]:
    """
    Récupère une 'page' de résultats depuis l'API.
    - offset : point de départ (pour la pagination)
    - limit  : nombre de lignes à demander
    Retour : liste de dictionnaires (lignes)
    """
    params = {
        "limit": limit,
        "offset": offset,
        "select": ", ".join(FIELDS),
        "order_by": "numac"   
    }
    r = requests.get(BASE_URL, params=params, timeout=60)
    r.raise_for_status()  
    return r.json().get("results", [])

# Exemple de test
sample = fetch_page(0, 10)
pd.DataFrame(sample).head()

HTTPError: 400 Client Error: Bad Request for url: https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/accidents-corporels-de-la-circulation-millesime/records/?limit=10&offset=0&limit=10&offset=0&select=numac%2C+departement%2C+codeinsee%2C+lumiere%2C+condition_atmospherique%2C+categorie_de_route%2C+type_de_collision%2C+lat%2C+date%2C+an%2C+mois%2C+jour%2C+hrmn&order_by=numac

In [ ]:
TARGET = 8000
PAGE = 100
DELAY = 0.3

all_rows = []
offset = 0
while offset < TARGET:
    chunk = fetch_page(offset, PAGE)
    if not chunk:
        break
    all_rows.extend(chunk)
    offset += PAGE
    time.sleep(DELAY)

# Sauvegarder un export "brut" pour traçabilité
df_raw = pd.DataFrame(all_rows)
df_raw.to_csv(RAW_DIR / "accidents_sample_raw.csv", index=False)
print(f"{len(df_raw)} lignes enregistrées")


HTTPError: 400 Client Error: Bad Request for url: https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/accidents-corporels-de-la-circulation-millesime/records/?limit=100&offset=0&select=numac%2C+departement%2C+codeinsee%2C+lumiere%2C+condition_atmospherique%2C+categorie_de_route%2C+type_de_collision%2C+lat%2C+date%2C+an%2C+mois%2C+jour%2C+hrmn&order_by=numac

In [ ]:
# Règles minimales : garder une clé, dédoublonner
df_clean = df_raw.copy()

# 1) supprimer les lignes sans identifiant
if "numac" in df_clean.columns:
    df_clean = df_clean.dropna(subset=["numac"]).drop_duplicates(subset=["numac"])

# 2) harmoniser des types simples
if "departement" in df_clean.columns:
    df_clean["departement"] = df_clean["departement"].astype(str)

# 3) renommer pour cohérence future
# df_clean = df_clean.rename(columns={
#     "condition_atmospherique": "meteo",
#     "categorie_de_route": "type_route"
# })

len(df_clean), df_clean.head(3)


(8000,
    numac departement codeinsee                      lumiere  \
 0      1          01     01053          Crépuscule ou aube   
 1      2          01     01053                   Plein jour   
 2      3          01     01248  Nuit sans éclairage public   
 
   condition_atmospherique     categorie_de_route  \
 0           Temps couvert         Voie Communale   
 1                 Normale  Route Départementale   
 2     Brouillard - fumée              Autoroute   
 
                                  type_de_collision   lat  
 0                   Deux véhicules – par le coté  None  
 1                 Deux véhicules – par l’arrière  None  
 2  Trois véhicules et plus - collisions multiples  None  )

In [ ]:
from pathlib import Path

CLEAN_DIR = Path("../data/cleaned")   # bon dossier
CLEAN_DIR.mkdir(parents=True, exist_ok=True)

clean_csv = CLEAN_DIR / "accidents_clean.csv"
df_clean.to_csv(clean_csv, index=False, encoding="utf-8")
clean_csv

WindowsPath('../data/cleaned/accidents_clean.csv')

## ✅ Prochaines étapes (après validation du modèle)

- Générer `dim_time` à partir des dates (ou d’un calendrier)  
- Créer `dim_location` (distinct de `codeinsee`, `departement`, lat/lon si dispo)  
- Créer `dim_conditions` (distinct de `lumiere`, `meteo`, `type_route`, `type_de_collision`)  
- Mapper les IDs (clé naturelle → clé de substitution)  
- Remplir `fact_accident` avec les mesures (`ttue`, `tbg`, `tbl`, `tindm`, `grav`)  
- Écrire `sql/schema.sql` et (option) charger via SQLAlchemy dans SQLite/MySQL/PostgreSQL